In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import tldextract
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv("Malicious-URLs.csv")
# Удаление дубликатов и пустых значений
data = data.drop_duplicates(subset="url").dropna()

# Проверка баланса классов
print("Распределение классов:")
print(data["label"].value_counts())
data

Распределение классов:
label
benign        10991
defacement     2727
phishing        912
malware         303
Name: count, dtype: int64


,count,url,label
0,0,br-icloud.com.br,phishing
1,1,mp3raid.com/music/krizz_kaliko.html,benign
2,2,bopsecrets.org/rexroth/cr/1.htm,benign
3,3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...,...
14995,14995,fortdefianceind.com/,benign
14996,14996,bookrags.com/lens/go.php?u=Hannah_Webster_Foster,benign
14997,14997,burbankairport.com/parking/buses-trains.html,benign
14998,14998,http://otomoto.pl/oferta/subaru-outback-legacy...,benign


In [14]:
data["label"] = data["label"].map({'defacement': 0, 'phishing': 1, 'benign': 2, 'malware': 3})
data

,count,url,label
0,0,br-icloud.com.br,1
1,1,mp3raid.com/music/krizz_kaliko.html,2
2,2,bopsecrets.org/rexroth/cr/1.htm,2
3,3,http://www.garage-pirenne.be/index.php?option=...,0
4,4,http://adventure-nicaragua.net/index.php?optio...,0
...,...,...,...
14995,14995,fortdefianceind.com/,2
14996,14996,bookrags.com/lens/go.php?u=Hannah_Webster_Foster,2
14997,14997,burbankairport.com/parking/buses-trains.html,2
14998,14998,http://otomoto.pl/oferta/subaru-outback-legacy...,2


In [15]:
def extract_features(url):
    features = {}
    try:
        # Лексические признаки
        features["url_length"] = len(url)
        features["num_special_chars"] = sum(url.count(c) for c in ['@', '%', '//', '?', '='])
        
        # Анализ домена
        ext = tldextract.extract(url)
        features["subdomain_count"] = len(ext.subdomain.split('.')) if ext.subdomain else 0
        features["domain_length"] = len(ext.domain)
        features["has_ip"] = 1 if any(part.isdigit() for part in ext.domain.split('.')) else 0
        
        # Семантические признаки
        keywords = ['login', 'admin', 'exe', 'php', 'config', 'secure']
        features["keyword_count"] = sum(1 for kw in keywords if kw in url)
        return features
    except Exception as e:
        print(f"Ошибка при обработке URL {url}: {e}")
        return None

# Применение функции
features = data["url"].apply(extract_features)
valid_indices = features[features.notna()].index
data = data.loc[valid_indices]
features = features.loc[valid_indices]

# Создание DataFrame с признаками
features_df = pd.DataFrame(features.tolist())
labels = data["label"].values

In [16]:
# Расчет веса для минорного класса (для параметра scale_pos_weight)
negative_count = sum(labels == 0)
positive_count = sum(labels == 1)
scale_pos_weight = negative_count / positive_count
print(f"\nВесовой коэффициент для минорного класса: {scale_pos_weight:.1f}")


Весовой коэффициент для минорного класса: 3.0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    features_df, 
    labels, 
    test_size=0.2, 
    stratify=labels, 
    random_state=42
)

In [27]:
# Создание DMatrix для XGBoost (оптимизированный формат данных)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Параметры модели
params = {
    "objective": "multi:softprob",  # Для мультикласса: "multi:softprob"
    "eval_metric": "mlogloss",# Для мультикласса: "mlogloss"
    "device": "cpu",
    "verbosity": 1,
    "booster": "gbtree",
    "eta": 0.05,                    # Learning rate
    "max_depth": 6,
    "scale_pos_weight": scale_pos_weight,  # Балансировка классов
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# Обучение с ранней остановкой
model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, "train"), (dtest, "test")],
    early_stopping_rounds=50,
    verbose_eval=50
)

XGBoostError: value 0 for Parameter num_class should be greater equal to 1
num_class: Number of output class in the multi-class classification.

In [19]:
# Предсказание
y_pred_proba = model.predict(dtest)
y_pred = (y_pred_proba > 0.5).astype(int)

# Метрики
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"\nROC-AUC: {roc_auc_score(y_test, y_pred_proba):.2f}")

# Важность признаков
xgb.plot_importance(model, importance_type="weight", max_num_features=10)
plt.title("Важность признаков (weight)")
plt.show()

NameError: name 'model' is not defined